In [4]:
from button_functions import Button_functions
from functions import Functions
from popups import Popups
from tkinter import *
import os
INFOFILE = 'information.json'

obj = Functions(INFOFILE)
func_but = Button_functions(INFOFILE)
pop = Popups(INFOFILE)
root, frm, labels = obj.rootInit()
if obj.checkInfofileInit():
    data = obj.getInfofileData()
phoneDir = data['Phone Directory']
titles = ["ben 10", "project zomboid", "the boys", "invincible", "Terror cannon", "Bestest day", "Spongebob"]

pc directory: PY_VAR4


In [5]:
showListbox = obj.gridListbox(
        frm,
        titles,
        r = 0,
        c = 0
    )


storeButton = obj.gridButton(
        frm,
        button_lab = "Store",
        button_function = lambda: func_but.testDelete(showListbox),
        r = 1,
        c = 0
)
root.mainloop()


<class 'tkinter.Listbox'>
('Spongebob', 'Bestest day', 'Terror cannon', 'invincible', 'the boys', 'project zomboid', 'ben 10')
ben 10
the boys
invincible
Terror cannon
Bestest day
Spongebob
<class 'tkinter.Listbox'>
('project zomboid',)
project zomboid
<class 'tkinter.Listbox'>
()
<class 'tkinter.Listbox'>
()


In [6]:
# TODO:
# Update the listbox after storing has happened

# make the showMap updateable